# Evaluating Coccinelle Semantic Patches in linux-mainline

### Setting up the notebook

In [1]:
# Import all needed libraries
import json
import calendar
import random
import os
import sys
from datetime import date, timedelta

import faker
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from delorean import parse
import matplotlib

# Enable inline plotting
%matplotlib inline

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.3 (v3.6.3:2c5fed8, Oct  3 2017, 18:11:49) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.20.3
Matplotlib version 2.1.0


### Preparing the data

In [3]:
# Dictionary listing all Semantic Patches in linux-4.14-rc6
# TODO: Should refactor into file 'cocci_linux_4_14_rc6.json'

cocci_linux_4_14_rc6 = {
    'info': {
        'repository': 'linux-4.14-rc6'
    },
    'scripts': [
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'alloc_cast.cocci',
            'confidence': 'High',
            'description': "Remove casting the values returned by memory allocation functions like "
                "kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. "
                "This makes an effort to find cases of casting of values returned "
                "by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, "
                "kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting "
                "as it is not required. The result in the patch case may need some reformatting."
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'kzalloc-simple.cocci',
            'confidence': 'High',
            'description': """use kzalloc rather than..."""
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'pool_zalloc-simple.cocci',
            'confidence': 'N.A.',
            'description': """Use *_pool_zalloc rather than..."""
        },
        # TODO: Complete fields for all the 59 entries
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'd_find_alias.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/debugfs',
            'filename': 'debugfs_simple_attr.cocci',
            'confidence': 'N.A.',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'drm-get-put.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'err_cast.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'kstrdup.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'memdup.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'memdup_user.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'platform_no_drv_owner.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'pm_runtime.cocci',
            'confidence': 'Medium',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'ptr_ret.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'resource_size.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'setup_timer.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'simple_open.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'vma_pages.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'clk_put.cocci',
            'confidence': 'Medium',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'devm_free.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'ifnullfree.cocci',
            'confidence': 'N.A.',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'iounmap.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'kfree.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'kfreeaddr.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'pci_free_consistent.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/free',
            'filename': 'device_node_continue.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/iterators',
            'filename': 'fen.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/iterators',
            'filename': 'ifnull.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/iterators',
            'filename': 'list_entry_update.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/iterators',
            'filename': 'use_after_iter.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/locks',
            'filename': 'call_kern.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/locks',
            'filename': 'double_lock.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/locks',
            'filename': 'flags.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/locks',
            'filename': 'mini_lock.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'array_size.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'badty.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'boolconv.cocci',
            'confidence': 'N.A.',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'boolinit.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'boolreturn.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'bugon.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'cond_no_effect.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'cstptr.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'doubleinit.cocci',
            'confidence': 'Low',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'ifaddr.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'ifcol.cocci',
            'confidence': 'Low',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'irqf_oneshot.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'noderef.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'of_table.cocci',
            'confidence': 'Medium',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'orplus.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'returnvar.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'semicolon.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/misc',
            'filename': 'warn.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/null',
            'filename': 'badzero.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/null',
            'filename': 'deref_null.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/null',
            'filename': 'eno.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/null',
            'filename': 'kmerr.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/tests',
            'filename': 'doublebitand.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/tests',
            'filename': 'doubletest.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/tests',
            'filename': 'odd_ptr_err.cocci',
            'confidence': 'High',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/tests',
            'filename': 'unsigned_lesser_than_zero.cocci',
            'confidence': 'Average',
            'description': "?"
        },
    ]
}

# cocci_linux_4_14_rc6

In [4]:
#print(cocci_linux_4_14_rc6)
len(cocci_linux_4_14_rc6['scripts'])

59

### Creating the DataFrame for cocci_linux_4_14_rc6

In [5]:
data = cocci_linux_4_14_rc6['scripts']
df = DataFrame(data) # , index=cocci_linux_4_14_rc6['scripts']['filename']

#df.count()
#df.describe()
#json.dumps(data)
#print(df.index)
#print()
#print(df.filename)
#print()

#df.head()

In [6]:
# Rearrange columns
cols = ['path', 'filename', 'confidence', 'description']
df = df[cols]

#df

In [7]:
# Notice that text of column "description" as shown above is truncated,
# however the cell inside the DataFrame contains the full value

#df.iloc[0]['description']

In [8]:
# Display full text for column "description"
# See <https://stackoverflow.com/questions/23388810/ipython-notebook-output-cell-is-truncating-contents-of-my-list>
# See <http://pandas.pydata.org/pandas-docs/stable/options.html>

from pandas import DataFrame
from IPython.display import HTML

pd.options.display.max_colwidth = 2000
pd.options.display.max_seq_items = 200

HTML(df.to_html())

,path,filename,confidence,description
0,scripts/coccinelle/api/alloc,alloc_cast.cocci,High,"Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting."
1,scripts/coccinelle/api/alloc,kzalloc-simple.cocci,High,use kzalloc rather than...
2,scripts/coccinelle/api/alloc,pool_zalloc-simple.cocci,N.A.,Use *_pool_zalloc rather than...
3,scripts/coccinelle/api,d_find_alias.cocci,Moderate,?
4,scripts/coccinelle/debugfs,debugfs_simple_attr.cocci,N.A.,?
5,scripts/coccinelle/api,drm-get-put.cocci,High,?
6,scripts/coccinelle/api,err_cast.cocci,High,?
7,scripts/coccinelle/api,kstrdup.cocci,High,?
8,scripts/coccinelle/api,memdup.cocci,High,?
9,scripts/coccinelle/api,memdup_user.cocci,High,?


In [9]:
# Export the dataframe to an Excel file
#df.to_excel('test.xls')

### Creating DataFrames for instrument-coccicheck reports 

In [10]:
# Report against linux-4.14-rc6 full-tree

report_linux_4_14_rc6_fulltree = {
    'info': {
        'note': """Report against linux-4.14-rc6 full-tree""",
        'script_start': 'Thu Oct  5 13:11:29 CEST 2017',
        'src_sha': 'd81fa669e3de7eb8a631d7d95dac5fbcb2bf9d4e',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',                'L':  13, 'W':   0, 'E':   0},
        {'filename': 'kzalloc-simple.cocci',            'L':   4, 'W':   4, 'E':   0},
        {'filename': 'pool_zalloc-simple.cocci',        'L':  22, 'W':  22, 'E':   0}, 
        {'filename': 'd_find_alias.cocci',              'L':   0, 'W':   0, 'E':   0},
        {'filename': 'debugfs_simple_attr.cocci',       'L':  83, 'W':  83, 'E':   0},
        {'filename': 'drm-get-put.cocci',               'L':  48, 'W':  48, 'E':   0},
        {'filename': 'err_cast.cocci',                  'L':   2, 'W':   2, 'E':   0},
        {'filename': 'kstrdup.cocci',                   'L':   0, 'W':   0, 'E':   0},
        {'filename': 'memdup.cocci',                    'L':   0, 'W':   0, 'E':   0},
        {'filename': 'memdup_user.cocci',               'L':   9, 'W':   9, 'E':   0},
        {'filename': 'platform_no_drv_owner.cocci',     'L':  20, 'W':   0, 'E':   0},
        {'filename': 'pm_runtime.cocci',                'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ptr_ret.cocci',                   'L':  55, 'W':  55, 'E':   0},
        {'filename': 'resource_size.cocci',             'L':  10, 'W':  10, 'E':   0},
        {'filename': 'setup_timer.cocci',               'L': 222, 'W':   0, 'E':   0},
        {'filename': 'simple_open.cocci',               'L':   1, 'W':   1, 'E':   0},
        {'filename': 'vma_pages.cocci',                 'L':   7, 'W':   7, 'E':   0},
        {'filename': 'clk_put.cocci',                   'L':   0, 'W':   0, 'E':   0},
        {'filename': 'devm_free.cocci',                 'L':  21, 'W':  21, 'E':   0},
        {'filename': 'ifnullfree.cocci',                'L': 124, 'W': 124, 'E':   0},
        {'filename': 'iounmap.cocci',                   'L':   5, 'W':   0, 'E':   5},
        {'filename': 'kfree.cocci',                     'L':  63, 'W':   0, 'E':  63},
        {'filename': 'kfreeaddr.cocci',                 'L':   0, 'W':   0, 'E':   0},
        {'filename': 'pci_free_consistent.cocci',       'L':   7, 'W':   0, 'E':   7},
        {'filename': 'device_node_continue.cocci',      'L':   3, 'W':   0, 'E':   3},
        {'filename': 'fen.cocci',                       'L':   2, 'W':   0, 'E':   2},
        {'filename': 'ifnull.cocci',                    'L':  27, 'W':   0, 'E':  27},
        {'filename': 'list_entry_update.cocci',         'L':   4, 'W':   0, 'E':   0},
        {'filename': 'use_after_iter.cocci',            'L':  27, 'W':   0, 'E':  27},
        {'filename': 'call_kern.cocci',                 'L':   6, 'W':   0, 'E':   6},
        {'filename': 'double_lock.cocci',               'L':  34, 'W':   0, 'E':   0},
        {'filename': 'flags.cocci',                     'L':   1, 'W':   0, 'E':   1},
        {'filename': 'mini_lock.cocci',                 'L': 288, 'W':   0, 'E':   0},
        {'filename': 'array_size.cocci',                'L':   8, 'W':   8, 'E':   0},
        {'filename': 'badty.cocci',                     'L':   4, 'W':   4, 'E':   0},
        {'filename': 'boolconv.cocci',                  'L': np.NaN, 'W': np.NaN, 'E': np.NaN},
        {'filename': 'boolinit.cocci',                  'L':   1775, 'W': np.NaN, 'E': np.NaN},
        {'filename': 'boolreturn.cocci',                'L': 113, 'W': 113, 'E':   0},
        {'filename': 'bugon.cocci',                     'L': 608, 'W': 304, 'E':   0},
        {'filename': 'cond_no_effect.cocci',            'L':  95, 'W':  95, 'E':   0},
        {'filename': 'cstptr.cocci',                    'L':   4, 'W':   0, 'E':   4},
        {'filename': 'doubleinit.cocci',                'L': 159, 'W':   0, 'E':   0},
        {'filename': 'ifaddr.cocci',                    'L':  13, 'W':   0, 'E':  13},
        {'filename': 'ifcol.cocci',                     'L': 110, 'W':   0, 'E':   0},
        {'filename': 'irqf_oneshot.cocci',              'L':  52, 'W':   0, 'E':  52},
        {'filename': 'noderef.cocci',                   'L':   6, 'W':   0, 'E':   6},
        {'filename': 'of_table.cocci',                  'L':   3, 'W':   0, 'E':   0},
        {'filename': 'orplus.cocci',                    'L': 223, 'W': 223, 'E':   0},
        {'filename': 'returnvar.cocci',                 'L': 254, 'W':   0, 'E':   0},
        {'filename': 'semicolon.cocci',                 'L': 327, 'W':   0, 'E':   0},
        {'filename': 'warn.cocci',                      'L':  18, 'W':   0, 'E':   0},
        {'filename': 'badzero.cocci',                   'L': 119, 'W': 119, 'E':   0},
        {'filename': 'deref_null.cocci',                'L':  32, 'W':   0, 'E':  32},
        {'filename': 'eno.cocci',                       'L':   1, 'W':   0, 'E':   1},
        {'filename': 'kmerr.cocci',                     'L':  13, 'W':   0, 'E':   0},
        {'filename': 'doublebitand.cocci',              'L':  79, 'W':   0, 'E':   0},
        {'filename': 'doubletest.cocci',                'L':  94, 'W':   0, 'E':   0},
        {'filename': 'odd_ptr_err.cocci',               'L':   4, 'W':   0, 'E':   0},
        {'filename': 'unsigned_lesser_than_zero.cocci', 'L':  18, 'W':  18, 'E':   0},
    ]
}

In [11]:
data2_full = DataFrame(report_linux_4_14_rc6_fulltree['statistics'])
data2_full = data2_full[['filename','L', 'W', 'E']]

data2_full

,filename,L,W,E
0,alloc_cast.cocci,13.0,0.0,0.0
1,kzalloc-simple.cocci,4.0,4.0,0.0
2,pool_zalloc-simple.cocci,22.0,22.0,0.0
3,d_find_alias.cocci,0.0,0.0,0.0
4,debugfs_simple_attr.cocci,83.0,83.0,0.0
5,drm-get-put.cocci,48.0,48.0,0.0
6,err_cast.cocci,2.0,2.0,0.0
7,kstrdup.cocci,0.0,0.0,0.0
8,memdup.cocci,0.0,0.0,0.0
9,memdup_user.cocci,9.0,9.0,0.0


In [12]:
#data2_full.plot()
#data2_full[['L', 'W', 'E']].plot()
#data2_full[['filename','L', 'W', 'E']].plot(kind='bar')

#len(data2_full['filename'].unique())

In [13]:
# Report against linux-4.4.50 minimized-tree

report_linux_4_4_50_minimizedtree = {
    'info': {
        'note': """Report against linux-4.4.50 minimzed-tree""",
        'script_start': 'Mon Oct 23 16:29:52 IST 2017',
        'cocci_sha': '9e66317d3c92ddaab330c125dfe9d06eee268aff',
        'src_sha': '90dcab23bbc81fbfa25dfdb91d4ce974a69bd210',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',                'L':   2, 'W':   0, 'E':   0},
        {'filename': 'kzalloc-simple.cocci',            'L':   0, 'W':   0, 'E':   0},
        {'filename': 'pool_zalloc-simple.cocci',        'L':   5, 'W':   0, 'E':   0}, 
        {'filename': 'd_find_alias.cocci',              'L':   0, 'W':   0, 'E':   0},
        {'filename': 'debugfs_simple_attr.cocci',       'L':   0, 'W':   0, 'E':   0},
        {'filename': 'drm-get-put.cocci',               'L':   0, 'W':   0, 'E':   0},
        {'filename': 'err_cast.cocci',                  'L':   0, 'W':   0, 'E':   0},
        {'filename': 'kstrdup.cocci',                   'L':   0, 'W':   0, 'E':   0},
        {'filename': 'memdup.cocci',                    'L':   0, 'W':   0, 'E':   0},
        {'filename': 'memdup_user.cocci',               'L':   4, 'W':   4, 'E':   0},
        {'filename': 'platform_no_drv_owner.cocci',     'L':   0, 'W':   0, 'E':   0},
        {'filename': 'pm_runtime.cocci',                'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ptr_ret.cocci',                   'L':   2, 'W':   2, 'E':   0},
        {'filename': 'resource_size.cocci',             'L':   0, 'W':   0, 'E':   0},
        {'filename': 'setup_timer.cocci',               'L':  11, 'W':   0, 'E':   0},
        {'filename': 'simple_open.cocci',               'L':   0, 'W':   0, 'E':   0},
        {'filename': 'vma_pages.cocci',                 'L':   2, 'W':   2, 'E':   0},
        {'filename': 'clk_put.cocci',                   'L':   0, 'W':   0, 'E':   0},
        {'filename': 'devm_free.cocci',                 'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ifnullfree.cocci',                'L':  14, 'W':  14, 'E':   0},
        {'filename': 'iounmap.cocci',                   'L':   1, 'W':   0, 'E':   1},
        {'filename': 'kfree.cocci',                     'L':   2, 'W':   0, 'E':   2},
        {'filename': 'kfreeaddr.cocci',                 'L':   0, 'W':   0, 'E':   0},
        {'filename': 'pci_free_consistent.cocci',       'L':   0, 'W':   0, 'E':   0},
        {'filename': 'device_node_continue.cocci',      'L':   0, 'W':   0, 'E':   0},
        {'filename': 'fen.cocci',                       'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ifnull.cocci',                    'L':   5, 'W':   0, 'E':   5},
        {'filename': 'list_entry_update.cocci',         'L':   1, 'W':   0, 'E':   0},
        {'filename': 'use_after_iter.cocci',            'L':   1, 'W':   0, 'E':   1},
        {'filename': 'call_kern.cocci',                 'L':   3, 'W':   0, 'E':   3},
        {'filename': 'double_lock.cocci',               'L':  11, 'W':   0, 'E':   0},
        {'filename': 'flags.cocci',                     'L':   0, 'W':   0, 'E':   0},
        {'filename': 'mini_lock.cocci',                 'L':  45, 'W':   0, 'E':   0},
        {'filename': 'array_size.cocci',                'L':   0, 'W':   0, 'E':   0},
        {'filename': 'badty.cocci',                     'L':   0, 'W':   0, 'E':   0},
        {'filename': 'boolconv.cocci',                  'L':   2, 'W':   2, 'E':   0},
        {'filename': 'boolinit.cocci',                  'L':  37, 'W':  37, 'E':   0},
        {'filename': 'boolreturn.cocci',                'L':   4, 'W':   4, 'E':   0},
        {'filename': 'bugon.cocci',                     'L':  92, 'W':  46, 'E':   0},
        {'filename': 'cond_no_effect.cocci',            'L':   1, 'W':   1, 'E':   0},
        {'filename': 'cstptr.cocci',                    'L':   1, 'W':   0, 'E':   1},
        {'filename': 'doubleinit.cocci',                'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ifaddr.cocci',                    'L':   0, 'W':   0, 'E':   0},
        {'filename': 'ifcol.cocci',                     'L':   3, 'W':   0, 'E':   0},
        {'filename': 'irqf_oneshot.cocci',              'L':   2, 'W':   0, 'E':   2},
        {'filename': 'noderef.cocci',                   'L':   1, 'W':   0, 'E':   1},
        {'filename': 'of_table.cocci',                  'L':   0, 'W':   0, 'E':   0},
        {'filename': 'orplus.cocci',                    'L':   5, 'W':   5, 'E':   0},
        {'filename': 'returnvar.cocci',                 'L':   8, 'W':   0, 'E':   0},
        {'filename': 'semicolon.cocci',                 'L':   4, 'W':   0, 'E':   0},
        {'filename': 'warn.cocci',                      'L':   5, 'W':   0, 'E':   0},
        {'filename': 'badzero.cocci',                   'L':   0, 'W':   0, 'E':   0},
        {'filename': 'deref_null.cocci',                'L':   0, 'W':   0, 'E':   0},
        {'filename': 'eno.cocci',                       'L':   0, 'W':   0, 'E':   0},
        {'filename': 'kmerr.cocci',                     'L':   0, 'W':   0, 'E':   0},
        {'filename': 'doublebitand.cocci',              'L':  20, 'W':   0, 'E':   0},
        {'filename': 'doubletest.cocci',                'L':   2, 'W':   0, 'E':   0},
        {'filename': 'odd_ptr_err.cocci',               'L':   0, 'W':   0, 'E':   0},
        {'filename': 'unsigned_lesser_than_zero.cocci', 'L':   0, 'W':   0, 'E':   0},
    ]
}

In [14]:
data3_minimized = DataFrame(report_linux_4_4_50_minimizedtree['statistics'])
data3_minimized = data3_minimized[['filename','L', 'W', 'E']]

data3_minimized

,filename,L,W,E
0,alloc_cast.cocci,2,0,0
1,kzalloc-simple.cocci,0,0,0
2,pool_zalloc-simple.cocci,5,0,0
3,d_find_alias.cocci,0,0,0
4,debugfs_simple_attr.cocci,0,0,0
5,drm-get-put.cocci,0,0,0
6,err_cast.cocci,0,0,0
7,kstrdup.cocci,0,0,0
8,memdup.cocci,0,0,0
9,memdup_user.cocci,4,4,0


### Assigning estimated safety relevance to cocci scripts

In [15]:
my_safetyrel_cocci = {
    'info': {
        # TODO
    },
    # rel: Safety Relevance (1=Highest, 9=lowest)
    'values': [
        {'filename': 'alloc_cast.cocci',                'rel': 5, 'notes': ''},
        {'filename': 'kzalloc-simple.cocci',            'rel': 5, 'notes': ''},
        {'filename': 'pool_zalloc-simple.cocci',        'rel': 8, 'notes': ''},
        {'filename': 'd_find_alias.cocci',              'rel': 3, 'notes': ''},
        {'filename': 'debugfs_simple_attr.cocci',       'rel': 6, 'notes': ''},
        {'filename': 'drm-get-put.cocci',               'rel': 6, 'notes': ''},
        {'filename': 'err_cast.cocci',                  'rel': 5, 'notes': ''},
        {'filename': 'kstrdup.cocci',                   'rel': 4, 'notes': ''},
        {'filename': 'memdup.cocci',                    'rel': 4, 'notes': ''},
        {'filename': 'memdup_user.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'platform_no_drv_owner.cocci',     'rel': 5, 'notes': ''},
        {'filename': 'pm_runtime.cocci',                'rel': 4, 'notes': ''},
        {'filename': 'ptr_ret.cocci',                   'rel': 5, 'notes': ''},
        {'filename': 'resource_size.cocci',             'rel': 4, 'notes': ''},
        {'filename': 'setup_timer.cocci',               'rel': 6, 'notes': ''},
        {'filename': 'simple_open.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'vma_pages.cocci',                 'rel': 4, 'notes': ''},
        {'filename': 'clk_put.cocci',                   'rel': 4, 'notes': ''},
        {'filename': 'devm_free.cocci',                 'rel': 2, 'notes': 'Beware of false positives'},
        {'filename': 'ifnullfree.cocci',                'rel': 7, 'notes': ''},
        {'filename': 'iounmap.cocci',                   'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'kfree.cocci',                     'rel': 2, 'notes': 'Beware of false positives'},
        {'filename': 'kfreeaddr.cocci',                 'rel': 1, 'notes': ''},
        {'filename': 'pci_free_consistent.cocci',       'rel': 4, 'notes': ''},
        {'filename': 'device_node_continue.cocci',      'rel': 3, 'notes': ''},
        {'filename': 'fen.cocci',                       'rel': 5, 'notes': ''},
        {'filename': 'ifnull.cocci',                    'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'list_entry_update.cocci',         'rel': 4, 'notes': ''},
        {'filename': 'use_after_iter.cocci',            'rel': 3, 'notes': 'Beware of false positives'},
        {'filename': 'call_kern.cocci',                 'rel': 4, 'notes': ''},
        {'filename': 'double_lock.cocci',               'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'flags.cocci',                     'rel': 2, 'notes': ''},
        {'filename': 'mini_lock.cocci',                 'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'array_size.cocci',                'rel': 4, 'notes': ''},
        {'filename': 'badty.cocci',                     'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'boolconv.cocci',                  'rel': 9, 'notes': 'Cleanup code only'},
        {'filename': 'boolinit.cocci',                  'rel': 8, 'notes': ''},
        {'filename': 'boolreturn.cocci',                'rel': 6, 'notes': 'Does not affect code behaviour'},
        {'filename': 'bugon.cocci',                     'rel': 8, 'notes': ''},
        {'filename': 'cond_no_effect.cocci',            'rel': 4, 'notes': 'Developed by Nicholas Mc Guire, OSADL'},
        {'filename': 'cstptr.cocci',                    'rel': 1, 'notes': ''},
        {'filename': 'doubleinit.cocci',                'rel': 7, 'notes': 'High rate of false positives'},
        {'filename': 'ifaddr.cocci',                    'rel': 1, 'notes': ''},
        {'filename': 'ifcol.cocci',                     'rel': 7, 'notes': 'High rate of false positives'},
        {'filename': 'irqf_oneshot.cocci',              'rel': 6, 'notes': ''},
        {'filename': 'noderef.cocci',                   'rel': 2, 'notes': ''},
        {'filename': 'of_table.cocci',                  'rel': 2, 'notes': ''},
        {'filename': 'orplus.cocci',                    'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'returnvar.cocci',                 'rel': 7, 'notes': ''},
        {'filename': 'semicolon.cocci',                 'rel': 8, 'notes': ''},
        {'filename': 'warn.cocci',                      'rel': 4, 'notes': ''},
        {'filename': 'badzero.cocci',                   'rel': 5, 'notes': ''},
        {'filename': 'deref_null.cocci',                'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'eno.cocci',                       'rel': 2, 'notes': ''},
        {'filename': 'kmerr.cocci',                     'rel': 2, 'notes': ''},
        {'filename': 'doublebitand.cocci',              'rel': 7, 'notes': 'Beware of false positives'},
        {'filename': 'doubletest.cocci',                'rel': 7, 'notes': 'Beware of false positives'},
        {'filename': 'odd_ptr_err.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'unsigned_lesser_than_zero.cocci', 'rel': 3, 'notes': ''},
        # TODO: Complete with all the 59 entries
    ]
}

In [16]:
data5_relevance = DataFrame(my_safetyrel_cocci['values'])
data5_relevance = data5_relevance[['filename','rel', 'notes']]

data5_relevance

,filename,rel,notes
0,alloc_cast.cocci,5,
1,kzalloc-simple.cocci,5,
2,pool_zalloc-simple.cocci,8,
3,d_find_alias.cocci,3,
4,debugfs_simple_attr.cocci,6,
5,drm-get-put.cocci,6,
6,err_cast.cocci,5,
7,kstrdup.cocci,4,
8,memdup.cocci,4,
9,memdup_user.cocci,4,


### Merge tables and aggregate results

In [17]:
#df

In [18]:
# Reference: <https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html>
#
# See also tutorial: Merge and Join dataframes with Panda
# <https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/>

# Merge data5_relevance and df on filename
result = pd.merge(data5_relevance,
                  df,
                  on='filename')

# result

In [19]:
def get_aggregate_LWE(dataframe, index):
    """Support function for df_append_aggregate_LWE"""
    # print('DEBUG: get_aggregate_LWE(dataframe={0}, index={1})'.format(id(dataframe), index))
    try:
        this_l = int(dataframe['L'][index])
    except ValueError:
        this_l = '?'
    # print('DEBUG: this_l=', this_l)
    try:
        this_w = int(dataframe['W'][index])
    except ValueError:
        this_w = '?'
    # print('DEBUG: this_w=', this_w)
    try:
        this_e = int(dataframe['E'][index])
    except ValueError:
        this_e = '?'
    # print('DEBUG: this_e=', this_e)
    s = "{0}/{1}/{2}".format(this_l, this_w, this_e)
    return s

#int(data2_full['W'][1])
#print(get_aggregate_LWE(data2_full, 0))
#print(get_aggregate_LWE(data2_full, 3))
#data2_full.count()

def df_append_aggregate_LWE(dataframe, col_name):
    """Append to dataframe a new column col_name
    with strings 'L/W/E' created as the aggregation of columns L, W and E"""
    col = []
    for k in dataframe.index:
        #print("DEBUG: k=", k)
        entry = get_aggregate_LWE(dataframe, k)
        #print("DEBUG: entry=", entry)
        col.append(entry)
    #print("DEBUG: col=", col)
    dataframe[col_name] = col

         
# Create a new column 'LWE_full' with the aggregation of columns 'L'/'W'/'E'
df_append_aggregate_LWE(data2_full, "LWE_full")
#data2_full.head()
data2_full
#
# Join data2_full on filename
result = pd.merge(result, data2_full, on='filename')
#result

# Create a new column 'LWE_minimal' with the aggregation of columns 'L'/'W'/'E'
df_append_aggregate_LWE(data3_minimized, "LWE_minimized")
#data3_minimized.head()
data3_minimized
#
# Join data3_minimize on filename
result = pd.merge(result, data3_minimized, on='filename')
#result

In [20]:
#result.sort_values(by='rel')
#x = result.sort_values(by='rel')[['filename', 'rel', 'notes', 'path', 'confidence']] # , 'description'
x = result.sort_values(by='rel')[['path', 'filename', 'rel', 'confidence', 'LWE_full', 'LWE_minimized', 'notes']] # , 'description'

#type(x)    # ==> pandas.core.frame.DataFrame
#x

In [21]:
# See <https://stackoverflow.com/questions/25698448/how-to-embed-html-into-ipython-output>
from IPython.core.display import display, HTML
display(HTML('<h3>Result: Prioritized list of Coccinelle scripts</h3>'))
display(HTML(x.to_html()))

,path,filename,rel,confidence,LWE_full,LWE_minimized,notes
22,scripts/coccinelle/free,kfreeaddr.cocci,1,High,0/0/0,0/0/0,
42,scripts/coccinelle/misc,ifaddr.cocci,1,High,13/0/13,0/0/0,
40,scripts/coccinelle/misc,cstptr.cocci,1,High,4/0/4,1/0/1,
54,scripts/coccinelle/null,kmerr.cocci,2,High,13/0/0,0/0/0,
53,scripts/coccinelle/null,eno.cocci,2,High,1/0/1,0/0/0,
18,scripts/coccinelle/free,devm_free.cocci,2,Moderate,21/21/0,0/0/0,Beware of false positives
31,scripts/coccinelle/locks,flags.cocci,2,High,1/0/1,0/0/0,
46,scripts/coccinelle/misc,of_table.cocci,2,Medium,3/0/0,0/0/0,
45,scripts/coccinelle/misc,noderef.cocci,2,High,6/0/6,1/0/1,
21,scripts/coccinelle/free,kfree.cocci,2,Moderate,63/0/63,2/0/2,Beware of false positives


In [22]:
# Collect main results into an Excel spreadsheet
# See <https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html>

# TODO: Trying to create file "output.xlsx" returns ModuleNotFoundError
writer = pd.ExcelWriter('evaluate_sel_cocci.xls')
#x.to_excel?

# Write sheets
x.to_excel(writer, 'Summary')
df.to_excel(writer, 'Cocci scripts in linux 4.14-rc6')
data5_relevance.to_excel(writer, 'Estimated SEL relevance')
data2_full.to_excel(writer, 'report_linux_4_14_rc6_fulltree')
data3_minimized.to_excel(writer, 'report_linux_4_4_50_minimized')

writer.save()